# Anderson Darling Test

The Anderson-Darling Test is used to determine whether a collection of data points follows a given distribution. It gives more weight to the tail than the KS test does. Mathematically, it tests the null hypothesis

\begin{align}
H_0: \text{data follows the distribution }P(x)
\end{align}

against the alternative hypothesis

\begin{align}
H_a: \text{data does not follow the distribution }P(x)
\end{align}

with the test statistic being

\begin{align}
A^2 = - n - S
\end{align}

where

\begin{equation}
S = \sum_{i=1}^{n} \frac{(2i - 1)}{n}[ \ln(F(Y_{i})) + \ln( 1 - F(Y_{n + 1 - i}) ]
\end{equation}

$F$ being the cumulative distribution function, and

$Y_i$ being the ordered data points

Ok, cool. But what does this equation really mean? How does this ensure tail is given more weight more it is in KS test? Let's break it down using a story and some examples!

## Scenario

<!-- Let's say you want to model how much money customers spend in a visit to your website. One data scientist on your team suggests that this random variable (amount spent) follows a normal distribution. While another suggest this is more exponential. How do you determine which one more accurately models customer behavior? You can perform AD test to get an analytical proof for this. -->


<!-- Let's say you want to model how much time each user spends on a given article -->


Let's say you run a datacenter. You want to model how much time between mechanical failures (network cables, disks, cooling, etc.) so that you can schedule maintainence accordingly. One data scientist on your team suggests that this random variable (amount between failures) follows a normal distribution. While another suggest this is more exponential. How do you determine which one more accurately models the datacenter machines behavior? Well, that's the question that AD test can help you answer!

## Get Dataset

Let's pull the data that your datacenter maintainence person has been collecting, and see what it looks like.

In [1]:
import numpy as np
from scipy.stats import norm, lognorm

from plotly import graph_objects as go

np.random.seed(42)

In [2]:
# visualization helper
def visualize(X, fig=None, fig_title="Histogram", x_title=None, y_title="Number of Points", trace_name=None):
    # create new figure object. else add trace to existing
    if fig is None:
        fig = go.Figure()
    
    # create histogram
    fig.add_trace(
        go.Histogram(x=X, name=trace_name, histnorm="probability")
    )
    
    # add axis details
    fig.update_layout(
        title=fig_title,
        xaxis_title=x_title,
        yaxis_title=y_title,
    )
    
    return fig

In [3]:
# assume three scenarios - when data is small, medium, large
n_pts_small = 50
n_pts_medium = 1000
n_pts_large = 10_000

# generate lognormal samples | 2, 0.75 | 5, 0.5 | 0, 0.5
dataset_small = np.random.lognormal(
    mean=5,
    sigma=0.3,
    size=n_pts_small,
)

dataset_medium = np.random.lognormal(
    mean=5,
    sigma=0.3,
    size=n_pts_medium,
)

dataset_large = np.random.lognormal(
    mean=5,
    sigma=0.3,
    size=n_pts_large,
)

dataset_longtail = np.random.lognormal(
    mean=5,
    sigma=0.6,
    size=n_pts_large,
)

# plot probability density plots to compare the datasets
fig = visualize(
    dataset_small,
    fig_title="Distribution of Days b/w Failures",
    x_title="Days b/w Failures",
    y_title="Number of Instances with N Days b/w Failures",
    trace_name="Actual Data (small)"
)
fig = visualize(
    dataset_medium,
    fig=fig,
    fig_title="Distribution of Days b/w Failures",
    x_title="Days b/w Failures",
    y_title="Number of Instances with N Days b/w Failures",
    trace_name="Actual Data (medium)"
)
visualize(
    dataset_large,
    fig=fig,
    fig_title="Distribution of Days b/w Failures",
    x_title="Days b/w Failures",
    y_title="Number of Instances with N Days b/w Failures",
    trace_name="Actual Data (large)"
)
visualize(
    dataset_longtail,
    fig=fig,
    fig_title="Distribution of Days b/w Failures",
    x_title="Days b/w Failures",
    y_title="Number of Instances with N Days b/w Failures",
    trace_name="Actual Data (longtail)"
)
fig.show()

Hmm well, based on the distribution plot, it's hard to tell if the samples come from a normal or a lognormal distribution.

## Create MLE Estimates

Okay, so now we'll get the MLE estimates for the normal and lognormal distributions.

That is, first we will determine that, assuming the data came from normal, what would the $\mu$ and $\sigma$ of that distribution be? We will then run the AD test to check the test statistic to determine how likely it is that the data indeed came from the normal defined by that $\mu$ and $\sigma$.

Then, we will determine that, assuming the data came from lognormal, what would the $\mu$ and $\sigma$ of that distribution be? We will then run the AD test to check the test statistic to determine how likely it is that the data indeed came from the lognormal defined by that $\mu$ and $\sigma$.

### Normal Parameters

In [4]:
# MLE estimates for mu, sigma
dataset_small.mean(), dataset_small.std()

(144.19545675977767, 41.10583174514265)

In [5]:
# estimate parameters for normal
normal_mean = dataset_small.mean()
normal_std = dataset_small.std()

normal_samples = np.random.normal(
    loc=normal_mean,
    scale=normal_std,
    size=n_pts_small,
)

fig = visualize(
    normal_samples,
    fig=fig,
    fig_title="Distribution of Days b/w Failures",
    x_title="Days b/w Failures",
    y_title="Number of Instances with N Days b/w Failures",
    trace_name="Normal Estimate"
)
fig.show()

### Lognormal Parameters

Note that the MLE estimate we're using is biased estimate. But for the sake of simplicity, we'll use it anyway. [Spoiler]Furthermore, we'll see that AD test is able to give correct results _in_ _spite_ of having biased estimates.

In [6]:
# MLE estimates
np.log(dataset_small).mean(), np.sqrt(
    ((np.log(dataset_small) - np.log(dataset_small).mean())**2).mean()
)

(4.932357828423158, 0.2772854813617272)

In [7]:
# estimate parameters for normal
lognormal_mean = np.log(dataset_small).mean()
lognormal_sigma = np.sqrt(
    ((np.log(dataset_small) - np.log(dataset_small).mean())**2).mean()
)

lognormal_samples = np.random.lognormal(
    mean=lognormal_mean,
    sigma=lognormal_sigma,
    size=n_pts_small,
)

fig = visualize(
    lognormal_samples,
    fig=fig,
    fig_title="Distribution of Days b/w Failures",
    x_title="Days b/w Failures",
    y_title="Number of Instances with N Days b/w Failures",
    trace_name="Lognormal Estimate"
)
fig.show()

## Run Test

Time to get our hands dirty and calculate the test statistic!

with the test statistic being

\begin{align}
A^2 = - n - S
\end{align}

where

\begin{equation}
S = \sum_{i=1}^{n} \frac{(2i - 1)}{n}[ \ln(F(Y_{i})) + \ln( 1 - F(Y_{n + 1 - i}) ]
\end{equation}

In [8]:
def compute_ad_test_S(cdf, ordered_data):
    """
    :param cdf: cumulative distribution function of the assumed distribution  
    :type cdf: callable
    
    :param ordered_data: data points arranged in ascending order
    :type ordered_data: list of floats
    """
    # ln(F(Y_i))
    first_term = np.log(cdf(ordered_data))

    # ln(1 - F(Y_n+1-i))
    second_term = np.log(1 - cdf(ordered_data[::-1]))
    
    # (2i - 1)
    multiplier = 2 * np.arange(start=1, stop=len(ordered_data)+1, ) - 1
    
    ad_test_S = (multiplier * (first_term + second_term)).mean()
    return ad_test_S

In [9]:
# cdf of normal dist
norm_cdf = norm(loc=normal_mean, scale=normal_std).cdf
S_normal = compute_ad_test_S(norm_cdf, np.sort(dataset_small))
S_normal

-50.67070223858309

In [10]:
# test statistic
A2_normal = - n_pts_small - S_normal
A_normal = np.sqrt(A2_normal)
A2_normal, A_normal

(0.6707022385830896, 0.8189641253333931)

In [11]:
# cdf of lognormal dist
lognorm_cdf = lognorm(s=lognormal_sigma, scale=np.exp(lognormal_mean)).cdf
S_lognormal = compute_ad_test_S(lognorm_cdf, np.sort(dataset_small))
S_lognormal

-50.19749165791983

In [12]:
# test statistic
A2_lognormal = - n_pts_small - S_lognormal
A_lognormal = np.sqrt(A2_lognormal)
A2_lognormal, A_lognormal

(0.19749165791982648, 0.4444003351931977)

Great, so using the AD test we are actually able to tell that the samples are more likely to be from a lognormal distribution! And this is congruent with the domain knowledge - exponential distributions are very often used for model such behaviors.

But wait there's more! What if we continued to collect data for a few more weeks - how would that affect the AD test results? Or what if your distribution had longer tails?

In these next (optional / bonus) sections, we'll explore how much the sample size and heavy-tailed-ness (kurtosis) sways the results for the AD test.

### Effect of sample size

### Effect of tail

### Effect of distribution

### Why Are Tails Weighted More?

This answer gives a great idea - https://stats.stackexchange.com/a/403284

The spread in multiple datasets sampled from a distribution is more in the center than the tails. Therefore tails need to be weighed more.

# Conclusion

**Pros**
- more sensitive than KS test

**Cons**
- test statistic has to be calculated